In [1]:
import numpy as np
import re
import spacy
from spacy.lang.en import English
from zipfile import ZipFile

vocabularySize = 20000

nlp = spacy.load("en_core_web_sm")
tokenizer = English().Defaults.create_tokenizer(nlp)

mapping = dict()
with ZipFile("imdb_trn.zip", "r") as zip:
    index = 0
    for i in range(len(zip.infolist())):
        with zip.open(str(i).zfill(5) + ".txt") as file:
            text = file.read().decode("utf-8")
            text = re.sub("[\t\r\n]", " ", text)
            text = " ".join(text.split())
            for token in [ token.text for token in tokenizer(text) ]:
                key = token.lower()
                if (key not in mapping):
                    mapping[key] = set()
                if (file not in mapping[key]):
                    mapping[key].add(i)
        index = index + 1

members = dict()
for k,v in mapping.items():
    freq = len(v)
    if (freq not in members):
        members[freq] = []
    if (k not in members[freq]):
        members[freq].append(k)

tuples = sorted(members.items(), reverse = True)
if (vocabularySize > len(mapping)):
    vocabularySize = len(mapping)

selected = []
needed = vocabularySize
for i in range(len(tuples)):
    if (needed > 0):
        candidates = tuples[i][1]
        np.random.shuffle(candidates)
        count = len(candidates)
        if (count > needed):
            for j in range(needed):
                selected.append(candidates[j])
                needed = needed - 1
        else:
            for candidate in candidates:
                selected.append(candidate)
                needed = needed - 1
    else:
        break

output = open("vocabulary.dat", "w", encoding = "utf-8")
for token in selected:
    output.write(str(len(mapping[token])) + "\t" + token + "\n")
output.close()